# Nonlinear Equations

## Overview of Direct Methods

- Gauss elimination:
    - Initial form: Ax = b
    - Final form: Ux = c
- LU decomposition:
    - Initial form: Ax = b
    - Final form: LUx = b
    - Takeaway: A = LU
- Gauss-Jordan elimination
    - Initial form: Ax = b
    - Final form: Ix = c

## Gauss Elimination Method
1. Calculate for ratio between rows.
2. Subtract
3. Back substitution.

In [1]:
import numpy as np
import math

In [2]:
def gauss_elimination(a, b):

    n = len(b)
    x = np.zeros(n).astype(float)

    # iterates pivot row
    # pivot element: main diagonal element, hence why 'j' is the pivot
    for j in range(n):

        # iterates for each modified row (pivot + 1)
        for i in range(j + 1, n):
                ratio = a[i, j] / a[j, j]
                a[i, j:n] = a[i, j:n] - ratio * a[j, j:n]
                b[i] = b[i] - ratio * b[j]

    # back-substitution phase
    for i in range(n-1, -1, -1):
        sum_term = np.dot(a[i, i+1:n], x[i+1:n])
        x[i] = (b[i] - sum_term) / a[i, i]

    return x

In [3]:
alist = [[1, 1, 1], [1, 2, -1], [2, 1, 2]]
blist = [6, 2, 10]
a = np.array(alist).astype(float)
b = np.array(blist).astype(float)

print(gauss_elimination(a, b))

[1. 2. 3.]


# Forward and Backward Substitution Overview

In [4]:
def forward_substitution(a, b):
    n = len(b)
    y = np.zeros(n)

    y[0] = b[0]
    for i in range(1, n):
        sum_term = np.dot(a[i, 0:n], y[0:n])
        y[i] = (b[i] - sum_term) / a[i, i]

    return y

def backward_substitution(a, y):
    n = len(y)
    x = np.zeros(n)

    for i in range(n-1, -1, -1):
        sum_term = np.dot(a[i, i+1:n], y[i+1:n])
        x[i] = (y[i] - sum_term) / a[i, i]

    return x

## LU Decomposition (Doolittle's Decomposition)

- Find U using Gauss-elimination on A.
- Forward U to A = LU to find L.
- Forward L to Ly = b to find y, solve for L using forward substitution.
- Forward y to Ux = y fo find for x, solve for x using back substitution.

### L-matrix
- Stores the multiplier, or the 'ratio' of the Gaussian elimination process.
- Lij is multiplier used to eliminates Aij

In [5]:
def lu_decomposition(a, b):
    n = len(a)
    u = a.copy()
    l = np.eye(n)

    # LU factorization of A
    for j in range(n):
        for i in range(j+1, n):
            l[i, j] = u[i, j] / u[j, j]
            u[i, j:n] = u[i, j:n] - l[i, j] * u[j, j:n]
    
    # ly = b, find y using gauss_elimination(l, b)
    y = gauss_elimination(l, b)

    # ux = y, find x using gauss_elimination(u, y)
    x = gauss_elimination(u, y)

    return x


In [6]:
print(lu_decomposition(np.array([[1, 1, 1], [1, 2, -1], [2, 1, 2]]), np.array([6, 2, 10])))

[1. 2. 3.]


## Cholesky Decomposition

- A variation of LU Decomposition.
- Differs with Doolittle's decomposition in which L = U^T

In [7]:
def cholesky_factor(a, b):
    n = len(a)
    l = a.copy()
    for j in range (n):

        # checks if definite positive for element in the main-diagonal
        try:
            l[j, j] = math.sqrt(l[j, j] / np.dot(l[j, 0:j], l[j, 0:j]))
        except ValueError:
            print("Matriks tidak bisa diselesaikan dengan dekomposisi Cholesky")
            return l
        
        # perform forward-substitution
        for i in range(j+1, n):
            sum_term = np.dot(l[i, 0:j], l[j, 0:j])
            l[i, j] = (l[i, j] - sum_term) / l[j, j]
        
    # change upper-part to 0
    for j in range(1, n):
            a[0:j, j] = 0.0

    return l

In [8]:
def cholesky_sol(l, b):
    n = len(b)
    y = np.zeros(n)
    x = np.zeros(n)

    # for ly = b, forward-substitution
    y[0] = b[0]

    for j in range(1, n):
        sum_term = np.dot(a[j, 0:n], y[0:n])
        y[j] = (b[j] - sum_term) / l[j, j]
    
    # for ux = y, u = l_transpose so use back-substitution
    for j in range(n-1, -1, -1):
        sum_term = np.dot(a[j, j+1:n], x[j+1:n])
        x[j] = (y[j] - sum_term) / l[j, j]

    return x

In [9]:
def cholesky(a, b):
    l = cholesky_factor(a, b)
    x = cholesky_sol(l, b)
    return x